<a href="https://colab.research.google.com/github/olaviinha/NeuralImageSuperResolution/blob/master/SuperRes_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Neural Image Super-Resolution <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">ESRGAN</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralImageSuperResolution" target="_blank"><font color="#999" size="4">Github</font></a>

Colab for [JoeyBallentine's fork](https://github.com/JoeyBallentine/ESRGAN) of [BlueAmulet's fork](https://github.com/BlueAmulet/ESRGAN) of [ESRGAN](https://github.com/xinntao/ESRGAN), an implementation of [Enhanced Super-Resolution Generative Adversarial Networks](https://arxiv.org/abs/1809.00219) by Xintao Wang et al. This notebook includes a few additional features.

- Enter all file/directory paths relative to your Google Drive root. 
- `image_or_directory_to_upscale` may be a file path or a directory path. If a directory path is given, all images in the given directory will be processed. 
- Leave `output_dir` blank to save the processed superres image(s) in the same (`image_or_directory_to_upscale`) directory.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
force_setup = False

pip_packages = 'typer rich gmic'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

%cd /content/
!git clone https://github.com/olaviinha/ESRGAN.git Colab-ESRGAN
!git clone https://github.com/olaviinha/BlurDetection2.git
%cd "/content/Colab-ESRGAN"

dir_tmp = '/content/tmp/'
dir_mask = '/content/tmp/mask/'
dir_input = '/content/Colab-ESRGAN/input/'
dir_dejpeg = '/content/Colab-ESRGAN/dejpeg/'
dir_upscaled = '/content/Colab-ESRGAN/upscaled/'
dir_output = '/content/Colab-ESRGAN/output/'
dir_models = '/content/Colab-ESRGAN/models/'
create_dirs([dir_tmp, dir_input, dir_mask, dir_dejpeg, dir_upscaled, dir_output, dir_models])

#----------------------------------------

import requests, gmic, cv2
from google.colab import output
from google.colab.patches import cv2_imshow
from IPython.display import Image
from datetime import datetime
force_setup = False

downloaded_models = []
#----------------------------------------

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title # Upscale

image_or_directory_to_upscale = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}

#@markdown <hr size="1" color="#666">
#@markdown <small>&nbsp;</small>

#@markdown ### Advanced settings

# All models:
model = "ESRGAN_and_FatalPixels" #@param ["ESRGAN_and_FatalPixels", "ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "NMKD", "None"]
# Selected models:
# model = "PSNR" #@param ["ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "deJpeg", "NMKD", "DeJPEG_only"]
## #@markdown <small>If you selected ESRGAN_and_PSNR, select interpolation ratio. 0 = ESRGAN only, 100 = PSNR only</small>
## ESRGAN_PSNR_ratio = 0 #@param {type:"slider", min:0, max:100, step:1}
#@markdown <small>When two models are selected from model menu, select ratio in which they are used. 0 = first model, 100 = second model.</small>
interpolation = 15 #@param {type:"slider", min:0, max:100, step:1}
#@markdown <small>Remove JPEG artefacts by running DeJPEG model prior to upscaling.</small>
dejpeg = True #@param {type:"boolean"}
#@markdown <small>Add G'MIC _mightly details_ effect prior to upscaling.</small>
sharpen = False #@param {type:"boolean"}
#@markdown <small>Include used model(s) and interpolation in filename. Probably helpful if you test out different settings.</small>
models_in_filename = False #@param {type:"boolean"}
#@markdown <small>Input path if you want to use models from your Drive. Otherwise models are downloaded every time you run this cell, providing their hosting servers are up and running.</small>
local_models_dir = "" #@param {type:"string"}

#----------------------------------------------------------
# #@markdown <hr color="#666">

# #@markdown <small>Timestamp output files (YYYYMMDDHHIISS).</small>
# timestamp = False #@param {type:"boolean"}
# #@markdown ### Dev options
# blur_mask = False #@param {type:"boolean"}
# blur_threshold = 39 #@param {type:"slider", min:0, max:200, step:1}
# show_progress = True #@param {type:"boolean"}
# show_result = True #@param {type:"boolean"}
timestamp = False
blur_mask = False
show_progress = False
blur_threshold = 1
show_result = False
#----------------------------------------------------------

def dlfail():
  op(c.fail, 'Failed to download model, unable to proceed.')
  sys.exit()

use_local = False
if local_models_dir is not '' and os.path.isdir(drive_root+local_models_dir):
  mdir = fix_path(drive_root+local_models_dir)
  if os.path.isfile(mdir+'RRDB_ESRGAN_x4.pth'):
    !cp {mdir}RRDB_ESRGAN_x4.pth /content/Colab-ESRGAN/models/RRDB_ESRGAN_x4.pth
    downloaded_models.append('ESRGAN')
  if os.path.isfile(mdir+'RRDB_PSNR_x4.pth'):
    !cp {mdir}RRDB_PSNR_x4.pth  /content/Colab-ESRGAN/models/RRDB_PSNR_x4.pth
    downloaded_models.append('PSNR')
  if os.path.isfile(mdir+'4x_fatal_Anime_500000_G.pth'):
    !cp {mdir}4x_fatal_Anime_500000_G.pth /content/Colab-ESRGAN/models/4x_FatalAnime_500000_G.pth
    downloaded_models.append('FatalAnime')
  if os.path.isfile(mdir+'4x_FatalPixels_340000_G.pth'):
    !cp {mdir}4x_FatalPixels_340000_G.pth /content/Colab-ESRGAN/models/4x_FatalPixels.pth
    downloaded_models.append('FatalPixels')
  if os.path.isfile(mdir+'1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth'):
    !cp {mdir}1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth /content/Colab-ESRGAN/models/1x_DeJpeg_Fatality_PlusULTRA.pth
    downloaded_models.append('DeJpeg')
  use_local = True
else:

  dlmsg = 'Download required models...'
  # RRDB ESRGAN & RRDB PSNR by Xintao Wang, Ke Yu, Shixiang Wu, Jinjin Gu, Yihao Liu, Chao Dong, Yu Qiao, Chen Change Loy
  if 'ESRGAN' in model and 'ESRGAN' not in downloaded_models:
    op(c.title, dlmsg)
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O models/RRDB_ESRGAN_x4.pth
    if os.path.getsize('models/RRDB_ESRGAN_x4.pth') > 2000:
      downloaded_models.append('ESRGAN')
    else:
      dlfail()
  if 'PSNR' in model and 'PSNR' not in downloaded_models:
    op(c.title, dlmsg)
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O models/RRDB_PSNR_x4.pth
    if os.path.getsize('models/RRDB_PSNR_x4.pth') > 2000:
      downloaded_models.append('PSNR')
    else:
      dlfail()
  # fatal_anime by twittman
  if 'FatalAnime' in model and 'FatalAnime' not in downloaded_models:
    op(c.title, dlmsg)
    !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O models/4x_FatalAnime_500000_G.pth
    if os.path.getsize('models/4x_FatalAnime_500000_G.pth') > 2000:
      downloaded_models.append('FatalAnime')
    else:
      dlfail()
  # fatal_pixels twittman
  if 'FatalPixels' in model and 'FatalPixels' not in downloaded_models:
    op(c.title, dlmsg)
    !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O models/4x_FatalPixels.pth
    if os.path.getsize('models/4x_FatalPixels.pth') > 2000:
      downloaded_models.append('FatalPixels')
    else:
      dlfail()
  # deJpeg_Fatality_PlusULTRA by twittman
  if ('DeJpeg' in model or dejpeg is True) and 'DeJpeg' not in downloaded_models:
    op(c.title, dlmsg)
    !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O models/1x_DeJpeg_Fatality_PlusULTRA.pth
    if os.path.getsize('models/1x_DeJpeg_Fatality_PlusULTRA.pth') > 2000:
      downloaded_models.append('DeJpeg')
    else:
      dlfail()
  # 4x_NMKD-YandereNeo-Lite_320k by nmkd
  if 'NMKD' in model and 'NMKD' not in downloaded_models:
    op(c.title, dlmsg)
    !gdown --id 14lA-Ks5quxheNyVeXRvzeoSAOm6ISDHn -O "/content/Colab-ESRGAN/models/4x_NMKD-YandereNeo-Lite_320k.pth"
    if os.path.getsize('models/4x_NMKD-YandereNeo-Lite_320k.pth') > 2000:
      downloaded_models.append('NMKD')
    else:
      dlfail()
  # output.clear()

#----------------------------------------------------------

sharpen_before = True
sharpen_after = False

show_state = show_progress

if os.listdir(dir_input):
  clean_dirs([dir_input, dir_output, dir_mask, dir_dejpeg, dir_upscaled])

image_or_directory_to_upscale = drive_root+image_or_directory_to_upscale

is_dir = False
if os.path.isfile(image_or_directory_to_upscale):
  input_images = [image_or_directory_to_upscale]
if os.path.isdir(image_or_directory_to_upscale):
  input_images = list_images(image_or_directory_to_upscale)
  input_images.sort()
  is_dir = True

if output_dir == '' or output_dir == 'same':
  output_dir = path_dir(input_images[0])
else:
  output_dir = fix_path(drive_root+output_dir)
  if os.path.isdir(drive_root+output_dir):
    output_dir_images = list_iimages(drive_root+output_dir)
  else:
    create_dirs([output_dir])

uniq_id = gen_id()

model_1_part = 100-interpolation
model_2_part = interpolation

# print('esrgan', model_1_part, 'fatalpx', model_2_part)

if model == 'ESRGAN_and_FatalPixels':
  # 25 75
  model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|4x_FatalPixels.pth@'+str(model_2_part)
if model == 'ESRGAN_and_PSNR':
  # 70 30
  model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|RRDB_PSNR_x4.pth@'+str(model_2_part)
if model == 'ESRGAN':
  model_file = 'RRDB_ESRGAN_x4.pth'
if model == 'PSNR':
  model_file = 'RRDB_PSNR_x4.pth'
if model == 'FatalAnime':
  model_file = '4x_FatalAnime_500000_G.pth'
if model == 'FatalPixels':
  model_file = '4x_FatalPixels.pth'
if model == 'DeJpeg':
  model_file = '1x_DeJpeg_Fatality_PlusULTRA.pth'
if model == 'NMKD':
  model_file = 'NMKD-YandereNeo_lite_320k_x4.pth'

op(c.title, 'Run ID:', uniq_id)
op(c.title, 'Images:', len(input_images))
if use_local is True:
  print('Using models from local dir', local_models_dir)
print('\n')

exists = []
broken = []
skipped = []

for input_image in input_images:

  clean_dirs([dir_input, dir_output, dir_mask, dir_dejpeg, dir_upscaled])
  if models_in_filename:
    final_file = output_dir+basename(input_image)+'_superres_'+uniq_id+'_'+model+'_'+str(model_1_part)+'_'+str(model_2_part)+'.png'
  else:
    final_file = output_dir+basename(input_image)+'_superres_'+uniq_id+'.png'

  existing = glob(output_dir+basename(input_image)+'_superres_*.png')
  incoming = output_dir+basename(input_image)
  file_exists = False
  input_image_d = input_image.replace(drive_root, '')

  for existing_item in existing:
    if incoming in existing_item:
      file_exists = True

  if file_exists:
    op(c.warn, 'Upscaled image seems to already exist. Skipping', input_image_d)
    exists.append(input_image_d)
    skipped.append(input_image_d)
  elif os.path.getsize(input_image) is 0:
    op(c.fail, 'Image is broken. Skipping', input_image_d)
    broken.append(input_image_d)
    skipped.append(input_image_d)
  else:

    state = input_image
    !cp "{input_image}" "{dir_input}"

    op(c.title, 'Processing image:', path_leaf(input_image))

    if dejpeg:
      op(c.okb, 'DeJpeg...')
      !python upscale.py 1x_DeJpeg_Fatality_PlusULTRA.pth --input "{dir_input}" --output "{dir_dejpeg}"
      state = dir_dejpeg+os.listdir(dir_dejpeg)[0]
      if show_state: Image(state)
      op(c.ok, 'Done.')

    ## Sharpen
    if sharpen and sharpen_before:
      op(c.okb, 'Sharpen...')
      tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
      gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
      # !cp "{tmp_file}" "{dir_input}"
      # state = input_file
      # state = dir_input+os.listdir(dir_input)[0]
      state = tmp_file
      if show_state: Image(state)
      op(c.ok, 'Done.\n')

    # print(input_image)
    # print(state)

    if state is not input_image:
      !cp "{state}" "{dir_input}"

    ## Neural upscale
    if model is not 'None' and model is not 'DeJPEG_only':
      op(c.okb, 'Upscale...')
      !python upscale.py "{model_file}" --input "{dir_input}" --output "{dir_upscaled}"
      # state = state.replace('input/', 'output/')
      state = dir_upscaled+os.listdir(dir_upscaled)[0]
      if show_state: Image(state)
      op(c.ok, 'Done.')

    ## Sharpen
    if sharpen and sharpen_after:
      op(c.okb, 'Sharpen...')
      tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
      gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
      state = tmp_file
      if show_state: Image(state)
      op(c.ok, 'Done.\n')

    !cp "{state}" "{final_file}"
    if os.path.isfile(final_file):
      op(c.ok, 'Superres file saved as:', final_file.replace(drive_root, '')+'\n')
    if show_result: Image(final_file)

    ## Blur map
    ##
    ## To handle blurred areas differently. Work in progress.
    ##
    if blur_mask:
      # blur_input_img = state
      blur_input_img = input_image
      blurmapSS = dir_tmp+'blur_mask_ss'
      blurmapSL = dir_tmp+'blur_mask_sl'
      blurmapBL = dir_tmp+'blur_mask_bl.png'
      %cd /content/BlurDetection2

      op(c.title, 'Generate blur map...')
      !python process.py -i "{blur_input_img}" -s {blurmapSS} -t "{blur_threshold}" -f
      state = blurmapSS+'_blur_map.png'
      if show_state: Image(state)
      op(c.ok, 'Done.\n')

      input_size = cv2.imread(state)
      width = int(input_size.shape[1])
      height = int(input_size.shape[0])
      dim = (width, height)
      temp_image = cv2.imread(state)
      upscaled = cv2.resize(temp_image, dim, interpolation = cv2.INTER_AREA)

      state = blurmapSL+'_blur_map.png'
      if show_state: Image(state)
      cv2.imwrite(state, upscaled)
      ba = str(width/200)
      gmic.run("-input "+blurmapSL+'_blur_map.png'+" -blur_linear "+ba+","+ba+",0 -glow 5 -adjust_colors 0,0,10,0,0 -adjust_colors 10,50,0,0,0 -output "+blurmapBL)
      if show_state: Image(blurmapBL)
      op(c.ok, 'Done.\n')

      !cp /content/tmp/blur_mask_bl.png /content/drive/MyDrive/ai/st3/test/
      !cp /content/tmp/blur_mask_sl_blur_map.png /content/drive/MyDrive/ai/st3/test/
      !cp /content/tmp/blur_mask_ss_blur_map.png /content/drive/MyDrive/ai/st3/test/
      !cp /content/tmp/blur_mask_ss_pretty_blur_map.png /content/drive/MyDrive/ai/st3/test/

if len(exists) > 0:
  op(c.warn, '\nImages that were not upscaled due to being previously upscaled and existing already:')
  for image in exists:
    print('-', image)

if len(broken) > 0:
  op(c.fail, '\nImages that were not upscaled due to being broken:')
  for image in broken:
    print('-', image)

op(c.title, '\nFIN.')